In [ ]:
import pandas as pd

#### Dimension Discounts

In [ ]:
# Lit et affiche le contenu de l'ancien fichier discount.csv

data = pd.read_csv('Old_Data/discounts.csv', sep=",")
data.head()

In [ ]:
# Récupération des champs utiles, suppression des doublons et génération de la clé primaire

df_Discounts = data[['Start', 'End', 'Discont', 'Category', 'Sub Category']]    
df_Discounts = df_Discounts.drop_duplicates()
df_Discounts = df_Discounts.reset_index(drop=True)
df_Discounts['Discount Key'] = df_Discounts.index +1

In [ ]:
# Création d'une copie du Dataframe pour supprimer les champs inutiles

df_Discounts_copy = df_Discounts.copy()
df_Discounts_copy.drop(['Category','Sub Category'], axis=1, inplace=True)

In [ ]:
# Affichage des données traitées

df_Discounts_copy.head()

In [ ]:
# Écriture des données traités dans un nouveau fichier discounts.csv

df_Discounts_copy.to_csv("Data/discounts.csv", index=False)

#### Dimension Stores

In [ ]:
# Lit et affiche le contenu de l'ancien fichier stores.csv

data = pd.read_csv('Old_Data/stores.csv', sep=",")
data.head()

In [ ]:
# Récupération des champs utiles et suppression des doublons

df_Stores = data[['Store ID', 'Store Name', 'City', 'Country', 'ZIP Code', 'Latitude', 'Longitude']]
df_Stores = df_Stores.drop_duplicates()

In [ ]:
# Affichage des données traitées

df_Stores.head()

In [ ]:
# Écriture des données traités dans un nouveau fichier stores.csv

df_Stores.to_csv("Data/stores.csv", index=False)

#### Dimension Products

In [ ]:
# Lit et affiche le contenu de l'ancien fichier products.csv

data = pd.read_csv('Old_Data/products.csv', sep=",")
data.head()

In [ ]:
# Récupération des champs utiles et suppression des doublons

df_Products = data[['Product ID','Category', 'Sub Category', 'Description FR', 'Sizes', 'Production Cost']]
df_Products = df_Products.drop_duplicates()

In [ ]:
# Affichage des données traitées

df_Products.head()

In [ ]:
# Écriture des données traités dans un nouveau fichier products.csv

df_Products.to_csv("Data/products.csv", index=False)

#### Dimension Date

In [ ]:
# Lit et affiche le contenu de l'ancien fichier transactions.csv

data = pd.read_csv('Old_Data/transactions.csv', sep=",")
data.head()

In [ ]:
# Récupération de la date, suppression des doublons et génération de la clé primaire

df_Date = data[['Date']] 
df_Date = df_Date.drop_duplicates()
df_Date = df_Date.reset_index(drop=True)
df_Date['Date Key'] = df_Date.index +1

In [ ]:
# Affichage des données traitées

df_Date.head()

In [ ]:
# Écriture des données traités dans un nouveau fichier date.csv

df_Date.to_csv("Data/date.csv", index=False)

#### Table de Fait

In [ ]:
# Récupération des champs utiles et suppression des doublons

df_Transactions = data[['Product ID', 'Store ID', 'Quantity', 'Unit Price', 'Line Total', 'Transaction Type', 'Invoice Total', 'Date']]    
df_Transactions = df_Transactions.drop_duplicates()

In [ ]:
# Affichage des données traitées

df_Transactions.head()

#### Réalisation des jointures

In [ ]:
# Jointure des Dataframes Transactions et Date dans la Table de Jointure

table_joint = pd.merge(df_Transactions, df_Date, on='Date', how='inner')
table_joint.head()

In [ ]:
# Jointure du DataFrame Products avec la table de Jointure et supression des champs inutiles

table_joint2 = pd.merge(table_joint, df_Products, on='Product ID', how='inner')
table_joint2.drop(['Description FR','Sizes', 'Production Cost'], axis=1, inplace=True)
table_joint2.head()

In [ ]:
# Jointure du DataFrame Discounts avec la table de Jointure
# Éxécution longue (60s)

table_joint3 = pd.merge(table_joint2, df_Discounts, on=['Category','Sub Category'], how='inner')
table_joint3.head()

In [ ]:
# Affichage de la taille de la table de Jointure

table_joint3.shape

In [ ]:
# Filtre du DataFrame de la table de Jointure pour ne garder que celle dont la date de la transaction appartient à la plage de la Promotion

df_Transactions = table_joint3[
    (table_joint3['Date'] >= table_joint3['Start']) &
    (table_joint3['Date'] <= table_joint3['End'])
]

In [ ]:
# Suppression de tout les champs qui ne sont plus utiles dans la table de Fait et Création de la clé primaire

df_Transactions.drop(['Date','Start', 'End', 'Discont', 'Category', 'Sub Category'], axis=1, inplace=True)
df_Transactions = df_Transactions.reset_index(drop=True)
df_Transactions['Transactions Key'] = df_Transactions.index +1
df_Transactions.head()

In [ ]:
# Affichage de la taille de la table de jointure pour constater la suppression de toute les lignes et colonnes inutiles

df_Transactions.shape

In [ ]:
# Écriture des données traités dans un nouveau fichier transactions.csv

df_Transactions.to_csv("Data/transactions.csv", index=False)